In [11]:
import pandas
import re

In [5]:
season = 2015
ms_teams_file = 'data/game_scores/teams_{}.csv'.format(season)
ms_teams = pandas.read_csv(ms_teams_file, sep=',').drop('Unnamed: 0', axis=1)

ms_teams[:5]

,name,team
0,Abilene_Chr,1
1,Air_Force,2
2,Akron,3
3,Alabama,4
4,Alabama_A&M,5


In [9]:
kg_teams_file = 'data/kaggle_2018/DataFiles/Teams.csv'
kg_teams = pandas.read_csv(kg_teams_file)
kg_teams[:5]

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [10]:
ms_teams.shape, kg_teams.shape

((351, 2), (364, 4))

In [22]:
def name_xform(name):
    team_name_map = {'UC Santa Barbara': 'Santa Barbara',
                 'Cal Poly': 'Cal Poly SLO',
                 'SIUE': 'Edwardsville'}
    
    name = re.sub('_', ' ', name)
    name = team_name_map.get(name, name)

    return name
    
    
ms_teams['TeamName'] = ms_teams['name'].apply(name_xform)
ms_teams[:5]

,name,team,TeamName
0,Abilene_Chr,1,Abilene Chr
1,Air_Force,2,Air Force
2,Akron,3,Akron
3,Alabama,4,Alabama
4,Alabama_A&M,5,Alabama A&M


In [23]:
combined_teams = ms_teams.merge(kg_teams[['TeamName', 'TeamID']], on='TeamName', how='left')
combined_teams[:5]

,name,team,TeamName,TeamID
0,Abilene_Chr,1,Abilene Chr,1101
1,Air_Force,2,Air Force,1102
2,Akron,3,Akron,1103
3,Alabama,4,Alabama,1104
4,Alabama_A&M,5,Alabama A&M,1105


In [24]:
# Check for ms_teams that are missing a kg_team
# Have to find aliases and add them to name_xform until none are missing
combined_teams[combined_teams['TeamID'].isnull()]

,name,team,TeamName,TeamID


In [25]:
# Check for kg_teams that are missing a ms_team
# Not usually a big deal - usually teams that were not D1 in the season in question
kg_teams[~kg_teams['TeamName'].isin(ms_teams['TeamName'])]

,TeamID,TeamName,FirstD1Season,LastD1Season
8,1109,Alliant Intl,1985,1991
17,1118,Armstrong St,1987,1987
20,1121,Augusta,1985,1991
27,1128,Birmingham So,2003,2006
33,1134,Brooklyn,1985,1992
46,1147,Centenary,1985,2011
114,1215,Hardin-Simmons,1985,1990
188,1289,Morris Brown,2002,2003
201,1302,NE Illinois,1991,1998
226,1327,Okla City,1985,1985


In [ ]:
def name_xform(name):
    team_name_map = {'UC Santa Barbara': 'Santa Barbara',
                 'Cal Poly': 'Cal Poly SLO',
                 'SIUE': 'Edwardsville'}
    
    name = re.sub('_', ' ', name)
    name = team_name_map.get(name, name)

    return name

In [39]:
combined_teams = list()

for season in range(2008, 2018 + 1):
    ms_teams_file = 'data/game_scores/teams_{}.csv'.format(season)
    ms_teams = pandas.read_csv(ms_teams_file, sep=',').drop('Unnamed: 0', axis=1)

    ms_teams['Season'] = season
    ms_teams['TeamName'] = ms_teams['name'].apply(name_xform)
    
    combined_teams.append(ms_teams.merge(kg_teams[['TeamName', 'TeamID']], on='TeamName', how='left'))

    print 'Teams missing in {}: {}'.format(season, combined_teams[-1][combined_teams[-1]['TeamID'].isnull()].shape[0])
    
teams = combined_teams[0].append(combined_teams[1:])
teams = teams.drop_duplicates().reset_index()

Teams missing in 2008: 0
Teams missing in 2009: 0
Teams missing in 2010: 0
Teams missing in 2011: 0
Teams missing in 2012: 0
Teams missing in 2013: 0
Teams missing in 2014: 0
Teams missing in 2015: 0
Teams missing in 2016: 0
Teams missing in 2017: 0
Teams missing in 2018: 0


In [44]:
teams.sort_values(['name', 'Season'])[:10]

,index,name,team,Season,TeamName,TeamID
2073,0,Abilene_Chr,1,2014,Abilene Chr,1101
2424,0,Abilene_Chr,1,2015,Abilene Chr,1101
2775,0,Abilene_Chr,1,2016,Abilene Chr,1101
3126,0,Abilene_Chr,1,2017,Abilene Chr,1101
3477,0,Abilene_Chr,1,2018,Abilene Chr,1101
0,0,Air_Force,1,2008,Air Force,1102
342,0,Air_Force,1,2009,Air Force,1102
689,0,Air_Force,1,2010,Air Force,1102
1036,0,Air_Force,1,2011,Air Force,1102
1381,0,Air_Force,1,2012,Air Force,1102


In [45]:
output_file = 'ms_to_kg_team_map.csv'
teams.sort_values(['name', 'Season']).to_csv(output_file, sep='|')